<a href="https://colab.research.google.com/github/bs21b038/assignment2data/blob/main/da6401_a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Transliteration system

import pandas as pd
import numpy as np

train_df= pd.read_csv('te.translit.sampled.train.tsv',sep='\t',header=None)
train_df.columns = ['devanagari','latin','serial']
train_df


dev_df= pd.read_csv('te.translit.sampled.dev.tsv',sep='\t',header=None)
dev_df.columns = ['devanagari','latin','serial']
dev_df

,devanagari,latin,serial
0,అంక,amka,1
1,అంక,anka,3
2,అంకం,amkam,1
3,అంకం,ankam,2
4,అంగీకరించ,amgiikarimcha,1
...,...,...,...
5678,హోమియో,hoemiyoe,1
5679,హోమియో,homeyo,1
5680,హోమేరిక్,homaric,1
5681,హోమేరిక్,homeric,1


In [5]:
seq=train_df['latin'][0]
seq

'amkita'

In [6]:
list(seq)
v=set()
print(v)
v.update(list(seq))
sorted(list(v))

set()


['a', 'i', 'k', 'm', 't']

In [7]:
#Vocab dictionaries
train_df=train_df.dropna()

def build_vocab(sequences):
    vocab = set()
    for seq in sequences:
        vocab.update(list(seq))
    vocab = sorted(list(vocab))
    vocab = ['<pad>', '<sos>', '<eos>'] + vocab  # special tokens
    char2idx = {ch: idx for idx, ch in enumerate(vocab)}
    idx2char = {idx: ch for ch, idx in char2idx.items()}
    return char2idx, idx2char

input_char2idx, input_idx2char = build_vocab(train_df["latin"])
target_char2idx, target_idx2char = build_vocab(train_df["devanagari"])

print("Input vocab size:", len(input_char2idx))
print("Target vocab size:", len(target_char2idx))

Input vocab size: 29
Target vocab size: 66


In [8]:
input_char2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28}

In [9]:
target_char2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 'ం': 3,
 'ః': 4,
 'అ': 5,
 'ఆ': 6,
 'ఇ': 7,
 'ఈ': 8,
 'ఉ': 9,
 'ఊ': 10,
 'ఋ': 11,
 'ఎ': 12,
 'ఏ': 13,
 'ఐ': 14,
 'ఒ': 15,
 'ఓ': 16,
 'ఔ': 17,
 'క': 18,
 'ఖ': 19,
 'గ': 20,
 'ఘ': 21,
 'చ': 22,
 'ఛ': 23,
 'జ': 24,
 'ఝ': 25,
 'ఞ': 26,
 'ట': 27,
 'ఠ': 28,
 'డ': 29,
 'ఢ': 30,
 'ణ': 31,
 'త': 32,
 'థ': 33,
 'ద': 34,
 'ధ': 35,
 'న': 36,
 'ప': 37,
 'ఫ': 38,
 'బ': 39,
 'భ': 40,
 'మ': 41,
 'య': 42,
 'ర': 43,
 'ఱ': 44,
 'ల': 45,
 'ళ': 46,
 'వ': 47,
 'శ': 48,
 'ష': 49,
 'స': 50,
 'హ': 51,
 'ా': 52,
 'ి': 53,
 'ీ': 54,
 'ు': 55,
 'ూ': 56,
 'ృ': 57,
 'ె': 58,
 'ే': 59,
 'ై': 60,
 'ొ': 61,
 'ో': 62,
 'ౌ': 63,
 '్': 64,
 '\u200c': 65}

In [10]:
# vectorial representations of letters -- integer based token encoding

def encode_sentence(sentence, char2idx, max_len, is_target=False):
    tokens = [char2idx[c] for c in sentence]
    if is_target:
        tokens = [char2idx["<sos>"]] + tokens + [char2idx["<eos>"]]
    return tokens + [char2idx["<pad>"]] * (max_len - len(tokens))

# Compute max lengths
max_encoder_seq_length = max(len(x) for x in train_df["latin"])
max_decoder_seq_length = max(len(y) for y in train_df["devanagari"]) + 2  # for <sos> and <eos>

print(max_encoder_seq_length)
print(max_decoder_seq_length)

25
22


In [11]:
def vectorize_data(df, input_char2idx, target_char2idx, max_encoder_len, max_decoder_len):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []

    for i in range(len(df)):
        input_seq = encode_sentence(df.iloc[i]["latin"], input_char2idx, max_encoder_len)
        target_seq = encode_sentence(df.iloc[i]["devanagari"], target_char2idx, max_decoder_len, is_target=True)

        decoder_input_seq = target_seq[:-1]
        decoder_target_seq = target_seq[1:]

        encoder_input_data.append(input_seq)
        decoder_input_data.append(decoder_input_seq)
        decoder_target_data.append(decoder_target_seq)

    return (
        np.array(encoder_input_data),
        np.array(decoder_input_data),
        np.array(decoder_target_data)
    )


In [12]:
# Prepare data for training/validation/testing
encoder_input_train, decoder_input_train, decoder_target_train = vectorize_data(
    train_df, input_char2idx, target_char2idx, max_encoder_seq_length, max_decoder_seq_length
)

encoder_input_val, decoder_input_val, decoder_target_val = vectorize_data(
    dev_df, input_char2idx, target_char2idx, max_encoder_seq_length, max_decoder_seq_length
)


In [17]:
print(encoder_input_train)
print(decoder_input_train)
print(decoder_target_train)

[[ 3 15 13 ...  0  0  0]
 [ 3 16 13 ...  0  0  0]
 [ 3 16 13 ...  0  0  0]
 ...
 [10 23 15 ...  0  0  0]
 [10 23 23 ...  0  0  0]
 [10 27 23 ...  0  0  0]]
[[ 1  5  3 ...  0  0  0]
 [ 1  5  3 ...  0  0  0]
 [ 1  5  3 ...  0  0  0]
 ...
 [ 1 51 64 ...  0  0  0]
 [ 1 51 64 ...  0  0  0]
 [ 1 51 64 ...  0  0  0]]
[[ 5  3 18 ...  0  0  0]
 [ 5  3 18 ...  0  0  0]
 [ 5  3 18 ...  0  0  0]
 ...
 [51 64 42 ...  0  0  0]
 [51 64 42 ...  0  0  0]
 [51 64 42 ...  0  0  0]]


In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.layers import Bidirectional


In [14]:
def get_rnn_cell(cell_type, hidden_units, return_sequences=True, return_state=True):
    if cell_type == 'RNN':
        return SimpleRNN(hidden_units, return_sequences=return_sequences, return_state=return_state)
    elif cell_type == 'LSTM':
        return LSTM(hidden_units, return_sequences=return_sequences, return_state=return_state)
    elif cell_type == 'GRU':
        return GRU(hidden_units, return_sequences=return_sequences, return_state=return_state)
    else:
        raise ValueError(f"Unknown cell type: {cell_type}")


In [ ]:
def build_seq2seq_model(input_vocab_size, target_vocab_size, embedding_dim, hidden_units,
                        encoder_layers=1, decoder_layers=1, cell_type='LSTM'):

    # Encoder
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    encoder_embedding = Embedding(input_vocab_size, embedding_dim, mask_zero=True, name='encoder_embedding')(encoder_inputs)

    encoder_outputs = encoder_embedding
    encoder_states = []

    for i in range(encoder_layers):
        # Return sequences True for all but last layer
        return_seq = (i != encoder_layers - 1)
        # RNN layer returns either 1 or 2 states (RNN/GRU vs LSTM)
        rnn_layer = get_rnn_cell(cell_type, hidden_units, return_sequences=return_seq, return_state=True)
        if cell_type == 'LSTM':
            encoder_outputs, state_h, state_c = rnn_layer(encoder_outputs)
            encoder_states = [state_h, state_c]
        else:
            encoder_outputs, state_h = rnn_layer(encoder_outputs)
            encoder_states = [state_h]

    # Decoder
    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    decoder_embedding = Embedding(target_vocab_size, embedding_dim, mask_zero=True, name='decoder_embedding')(decoder_inputs)

    decoder_outputs = decoder_embedding
    decoder_states = []

    for i in range(decoder_layers):
        return_seq = True  # Decoder must output full sequences for teacher forcing
        rnn_layer = get_rnn_cell(cell_type, hidden_units, return_sequences=return_seq, return_state=True)
        if cell_type == 'LSTM':
            decoder_outputs, state_h, state_c = rnn_layer(decoder_outputs, initial_state=encoder_states)
            decoder_states = [state_h, state_c]
        else:
            decoder_outputs, state_h = rnn_layer(decoder_outputs, initial_state=encoder_states)
            decoder_states = [state_h]

    # Output layer (project hidden state to vocab size)
    decoder_dense = Dense(target_vocab_size, activation='softmax', name='output_projection')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model


In [ ]:
for i in range(10):
  print(len(train_df['latin'][i]))